In [85]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib tk

In [2]:
# Define the Sphere function
def sphere(x):
    return np.sum(x**2)

# Gradient of the Sphere function (known)
def sphere_gradient(x):
    noise = np.random.randn(*x.shape)
    return 2 * x + noise * np.linalg.norm(x)

cond = 1e3
def ellipsoid(x, cond=cond):
    return sum(cond**(np.arange(len(x)) / (len(x) - 1 + 1e-9)) * np.asarray(x)**2)

def ellipsoid_gradient(x, cond=cond):
    return 2 * cond**(np.arange(len(x)) / (len(x) - 1 + 1e-9)) * np.asarray(x)

In [3]:
def evolution_strategy(f, x0, sigma, iterations=100, mu=5, lambda_=10):
    dim = len(x0)
    x = x0

    # Initialize path variable, decay and damping factors for Cumulative LeArning Rate Adaptation (CLARA)
    path = np.zeros(dim)
    c = 0.2
    d = 0.2
    cos_theta = 0

    # Initialize return variables
    candidate_solutions = []
    step_size = []
    path_norm = []
    step_norm = []
    cos_thetas = []

    weights = np.linspace(1, 2, mu)  # Assign higher weight to best individuals
    weights /= weights.sum()  # Normalize weights to sum to 1

    for i in range(iterations):
        # Generate offspring
        pop = np.random.randn(lambda_, dim)

        # Select mu best individuals
        selected = pop[np.argsort([f(x + sigma * ind) for ind in pop])[:mu]]

        # Update current solution
        step = np.sum(selected.T * weights, axis=1)
        x = x + sigma * step
        candidate_solutions.append(x)

        # Update mean estimate of cosine of the angle change between current and mean gradient
        cos_theta = (1 - c) * cos_theta + np.sqrt(c * (2 - c)) * np.dot(step, path) / (np.linalg.norm(path) * np.linalg.norm(step) + 1e-8)
        cos_thetas.append(cos_theta)

        # Update path
        path = (1 - c) * path + np.sqrt(c * (2 - c)) * step
        path_norm.append(np.linalg.norm(path)**2)
        step_norm.append(np.linalg.norm(step)**2)

        # Update step-size
        sigma = sigma * np.exp(d * ((np.linalg.norm(path)**2 / dim) - 1))
        step_size.append(sigma)


        print(f'Iteration {i}: current fitness = {f(x)}')

    return candidate_solutions, step_size, path_norm, step_norm, cos_thetas

In [4]:
def gradient_descent(gradient, x0, lr=0.1, iterations=100):
    """
    Performs gradient descent to minimize a function.

    :param gradient: Function that computes the gradient ∇f(x).
    :param x0: Initial guess (NumPy array).
    :param lr: Learning rate (step size).
    :param iterations: Number of iterations.
    :return: Final optimized value of x.
    """
    x = x0  # Initialize x
    dim = len(x0)  # Get dimension of search space

    # Initialize path variable, decay and damping factors for Cumulative LeArning Rate Adaptation (CLARA)
    path = np.zeros(dim)
    cos_theta = 0
    c = 0.2
    d = 1e-3

    # Initialize return variables
    candidate_solutions = []
    learning_rate = []
    path_norm = []
    gradient_norm = []
    cos_thetas = []

    for i in range(iterations):
        grad = gradient(x)  # Compute gradient
        x = x - lr * grad  # Update step

        # Update mean estimate of cosine of the angle change between current and mean gradient
        cos_theta = (1 - c) * cos_theta + np.sqrt(c * (2 - c)) * np.dot(grad, path) / (np.linalg.norm(path) * np.linalg.norm(grad) + 1e-8)

        # Update path
        path = (1 - c) * path + np.sqrt(c * (2 - c)) * grad

        # Update learning rate
        lr = lr * np.exp(d * (np.linalg.norm(path)**2 / dim - 1))

        candidate_solutions.append(x)
        learning_rate.append(lr)
        path_norm.append(np.linalg.norm(path)**2)
        gradient_norm.append(np.linalg.norm(grad)**2)
        cos_thetas.append(cos_theta)

    print('Optimized x: ', x)

    return candidate_solutions, learning_rate, path_norm, gradient_norm, cos_thetas

In [5]:
def estimate_adam_step_norm(n_steps=1, dim=2, beta1=0.9, beta2=0.999, num_samples=1000):
    """
    Estimates the expectation of the norm of the Adam step using Monte Carlo simulation.

    Parameters:
    n_steps (int): Number of steps to simulate for
    dim (int): Dimensionality of the vectors.
    beta1 (float): Adam optimizer's first moment decay rate.
    beta2 (float): Adam optimizer's second moment decay rate.
    num_samples (int): Number of Monte Carlo samples.

    Returns:
    float: Estimated expectation of the norm of step_adam.
    """

    # Generate samples and compute step_adam norm
    step_adam_norm = np.zeros(n_steps)

    for _ in range(num_samples):
        m = np.random.randn(dim)
        v = np.random.randn(dim)**2
        for i in range(n_steps):
            grad = np.random.randn(dim)  # i.i.d. standard normal vector
            m = beta1 * m + (1 - beta1) * grad
            m_hat = m / (1 - beta1**(i + 1))
            v = beta2 * v + (1 - beta2) * grad**2
            v_hat = v / (1 - beta2**(i + 1))
            step_adam = m_hat / (np.sqrt(v_hat) + 1e-8)
            step_adam_norm[i] += np.linalg.norm(step_adam)

    # Compute expectation
    return step_adam_norm / num_samples

In [10]:
norm_step_monte_carlo = estimate_adam_step_norm(n_steps=10000)

Text(0.5, 1.0, 'Monte Carlo estimate of $\\|\\mathbb{E}(\\hat{p}_t)\\|$')

In [89]:
# Create vertical subplots (2 rows, 1 column)
fig, axes = plt.subplots(2, 1, figsize=(8, 8))

# First subplot: Line plot of the values
axes[0].semilogy(norm_step_monte_carlo, color='blue', alpha=0.7)
axes[0].set_title(r'Monte Carlo estimate of $\|\mathbb{E}(\hat{p}_t)\|$')
axes[0].set_xlabel('Iteration')
axes[0].set_ylabel(r'$\|\mathbb{E}(\hat{p}_t)\|$')
axes[0].grid()

# Second subplot: KDE plot
sns.kdeplot(norm_step_monte_carlo, bw_adjust=0.5, ax=axes[1], color="red")
axes[1].set_title('KDE Estimate')
axes[1].set_xlabel(r'$\|\mathbb{E}(\hat{p}_t)\|$')
axes[1].set_ylabel('Density')
axes[1].grid()

# Adjust layout
plt.tight_layout()
plt.show()

In [93]:
def adam(gradient, x0, lr=0.1, iterations=100, adapt_lr=True):
    """
    Performs gradient descent to minimize a function.

    :param gradient: Function that computes the gradient ∇f(x).
    :param x0: Initial guess (NumPy array).
    :param lr: Learning rate (step size).
    :param iterations: Number of iterations.
    :return: Final optimized value of x.
    """
    x = x0  # Initialize x
    dim = len(x0)  # Get dimension of search space

    # Initialize path variable, decay and damping factors for Cumulative LeArning Rate Adaptation (CLARA)
    m = np.zeros(dim)
    v = np.zeros(dim)
    cos_theta = 0
    c = 0.2
    d = 1e-3
    beta1 = 0.9
    beta2 = 0.999

    # Estimate of average of cos(angle) between 2 Gaussian vectors
    # est_angle = average_angle_gaussian_vectors(dim)

    # Initialize return variables
    candidate_solutions = []
    learning_rate = []
    step_norm = []
    gradient_norm = []
    cos_thetas = []

    for i in range(iterations):
        grad = gradient(x)  # Compute gradient

        # Update step
        m = beta1 * m + (1 - beta1) * grad
        m_hat = m / (1 - beta1**(i + 1))
        v = beta2 * v + (1 - beta2) * grad**2
        v_hat = v / (1 - beta2**(i + 1))
        step_adam = m_hat / (np.sqrt(v_hat) + 1e-8)

        # Update solution
        x = x - lr * step_adam

        # Update mean estimate of cosine of the angle change between current and mean gradient
        cos_theta = (1 - c) * cos_theta + np.sqrt(c * (2 - c)) * np.dot(grad, step_adam) / (np.linalg.norm(step_adam) * np.linalg.norm(grad) + 1e-8)

        # Update learning rate
        if adapt_lr:
            lr = lr * np.exp(d * (np.linalg.norm(step_adam) / norm_step_monte_carlo[i] - 1))

        candidate_solutions.append(x)
        learning_rate.append(lr)
        step_norm.append(np.linalg.norm(step_adam))
        gradient_norm.append(np.linalg.norm(grad))
        cos_thetas.append(cos_theta)

    print('Optimized x: ', x)

    return candidate_solutions, learning_rate, step_norm, gradient_norm, cos_thetas

In [96]:
dim = 2
x0 = np.ones(dim)
lr0 = 1e-3
budget = 10000
adapt_lr = False

choice = 1

if choice == 0:  # Vanilla gradient descent
    optimizer = gradient_descent
    f = sphere_gradient
    fig_title = 'Vanilla gradient descent'  # '(mu, lambda)-ES'
elif choice == 1:  # Adam
    optimizer = adam
    f = sphere_gradient
    fig_title = 'Adam'
else:  # Evolution strategy
    optimizer = evolution_strategy
    f = sphere
    fig_title = '(mu, lambda)-ES'

candidate_sol, learning_rates, path_norm, grad_norm, cos_theta = optimizer(f, x0, lr0, iterations=budget, adapt_lr=adapt_lr)

Optimized x:  [ 1.66521074e-101 -2.53335197e-099]


In [97]:
results = [[np.linalg.norm(x) for x in candidate_sol],
           learning_rates,
           path_norm,
           grad_norm,
           [path_norm[i] / norm_step_monte_carlo[i] for i in range(len(path_norm))],  # grad_norm[i]
           cos_theta
           ]
fig_titles = ['Distance to optimum',
              'Learning rate',
              'Path norm',
              'Gradient/step norm',
              'Path-Monte Carlo estimate ratio',
              'Angle between path and current grad./step'
              ]
y_labels = [r'$\|x_t\|$',
            r'$\alpha_t$',
            r'$\|p_t\|$',
            r'$\|g_t\|$',
            r'$\|p_t\| / \|\hat{p}_t\|^2$',
            r'$\cos(\theta)$'
            ]
colors = ['r',
          'g',
          'b',
          'c',
          'm',
          'y'
          ]

fig, axes = plt.subplots(3, 2, figsize=(12, 12))  # 2 rows, 2 columns
fig.suptitle(f'{fig_title} on the noisy sphere, D = {str(len(x0))}, lr0 = {lr0:.1e}')

for i, ax in enumerate(axes.flat):  # Iterate over subplots
    if i == len(results) - 1:
        ax.plot(results[i], color=colors[i])
    else:
        ax.semilogy(results[i], color=colors[i])
    ax.set_title(fig_titles[i])
    ax.set_xlabel('Iterations')
    ax.set_ylabel(y_labels[i])
    ax.grid(True)

# Adjust layout and show
plt.tight_layout()
plt.show()